Feedrank on viettel comments
================

Dataset: 24K unique comments contain viettel, vietel,vt, etc.

Tasks:

- clustering the feeds
- classification based on instances: kNN
- prediction of cluster for each feed

Steps:

- how to represent feed: tfidf, lda, doc2vec 100?
- how to measure similarity: cosine, jsd?
- how to choose number of clusters?


LR: 0.6, SVC: 0.4
Mega Combinator:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     20377
        1.0       0.98      0.98      0.98      3616

avg / total       0.99      0.99      0.99     23993

Mega Classifier: 0.99 AUC

In [1]:
import re 
import string
import unicodedata
import sys

STOPWORDS = [u"ấy", u"bị", u"bởi", u"cả", u"các", u"cái", u"cần", u"càng", u"chỉ", u"chiếc", u"cho", u"chứ", u"chưa", 
             u"chuyện", u"có", u"có_thể", u"cứ", u"của", u"cùng", u"cũng", u"đã", u"đang", u"đây", u"để", u"đến_nỗi", 
             u"đều", u"điều", u"do", u"đó", u"được", u"dưới", u"gì", u"khi", u"không", u"là", u"lại", u"lên", u"lúc", 
             u"mà", u"mỗi", u"một cách", u"này", u"nên", u"nếu", u"ngay", u"nhiều", u"như", u"nhưng", u"những", u"nơi", 
             u"nữa", u"phải", u"qua", u"ra", u"rằng", u"rất", u"rồi", u"sau", u"sẽ", u"so", u"sự", u"tại", u"theo", 
             u"thì", u"trên", u"trước", u"từ", u"từng", u"và", u"vẫn", u"vào", u"vậy", u"vì", u"việc", u"với", u"vừa",
             u"_num", u"wwdateww", u"wwtimeww", u"wwemailww", u"wwipww", u"wwurlww", u"wwnumberww"
            ]

tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P') and i != 45 and i!= 95)

def vi_trans_unicode(su):
    return su.translate(tbl)

def vi_strip_text2(s):
    s = re.sub(r"&amp;", "", s)
    s = vi_trans_unicode(s)
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub(r"__+", "_", s)
    s = re.sub(r"--+", "-", s)
    s = re.sub(r'(.)\1+', r'\1\1', s)
    s = ' '.join([w if not w[0].isdigit() else u"wwNUMBERww" for w in s.strip().split()])
    return s


def vi_clean3(line):
    words = line.replace('.','').strip().split()
    words = [w.lower() for w in words]
    return ' '.join(words)

def vi_remove_stop_1char(line):
    words = line.split()
    words = [w for w in words if w not in STOPWORDS and len(w) > 1]
    return ' '.join(words)

import requests
import codecs
import json

tok_url = "http://192.168.0.215:8081/api/v1.0/document/filter"

def get_tokens(tok_url, data):
    try:
        rq = requests.post(tok_url, data=data.encode('utf-8'))
        if rq.content:
            tok_doc = ' '.join(json.loads(rq.content)['sentences'])
        else:
            tok_doc = None
        return tok_doc
    except Exception, e:
        print e

## TFIDF

In [ ]:
with codecs.open("/home/laampt/nlp/data/feeds/tok/FE.feed.txt.nodup.word.tok3", encoding="utf-8", mode="r") as fin:
    feed_raw = fin.readlines()
    
print len(feed_raw)
print feed_raw[0]

In [ ]:
feed_clean = []
# pos_posts_io = []
for i, raw in enumerate(feed_raw):
    if i % 100 == 0: print "[*] Progress ", i
    doc = vi_remove_stop_1char(vi_clean3(vi_strip_text2(raw)))
    feed_clean.append(doc)

print len(feed_clean)
print feed_clean[0]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer_tfidf = TfidfVectorizer(min_df=2, 
                                   max_df=0.95,
                                   sublinear_tf=True,
                                   analyzer = "word",   
                                   tokenizer = None,    
                                   preprocessor = None, 
                                   stop_words = None,   
                                   max_features = None
                                )

feed_feature_tfidf = vectorizer_tfidf.fit_transform(feed_clean)

In [ ]:
print vectorizer_tfidf

In [ ]:
print feed_feature_tfidf[0]

In [ ]:
feed_feature_tfidf = feed_feature_tfidf.toarray()

In [ ]:
X = feed_feature_tfidf.toarray().copy()

print X.shape

In [ ]:
print X[0]

## Kmean

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(X)
PCA(copy=True, n_components=2, whiten=True)
print(pca.explained_variance_ratio_) 

In [ ]:
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans

In [ ]:
kmeans = KMeans(n_clusters=5, n_jobs=4).fit(X)

In [ ]:
mbk = MiniBatchKMeans(init='k-means++', n_clusters=5, batch_size=2000, n_init=10, max_no_improvement=100, verbose=0)
mbk.fit(X)

In [ ]:
print mbk.inertia_

In [ ]:
mbk_means_labels = mbk.labels_
mbk_means_cluster_centers = mbk.cluster_centers_
mbk_means_labels_unique = np.unique(mbk_means_labels)

In [ ]:
print mbk_means_labels
print
print mbk_means_cluster_centers
print
print mbk_means_labels_unique

In [ ]:
mbk2 = MiniBatchKMeans(init='k-means++', n_clusters=20, batch_size=2000, n_init=10, max_no_improvement=100, verbose=0)
mbk2.fit(X)

In [ ]:
print mbk2.inertia_

In [ ]:
mbk50 = MiniBatchKMeans(init='k-means++', n_clusters=50, batch_size=2000, n_init=10, max_no_improvement=100, verbose=0)
mbk50.fit(X)

In [ ]:
print mbk50.inertia_

## Export

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(mbk, "/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_mbk5.pkl")
joblib.dump(mbk2, "/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_mbk20.pkl")
joblib.dump(mbk50, "/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_mbk50.pkl")

In [ ]:
joblib.dump(vectorizer_tfidf, "/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_vectorizer_tfidf.pkl")

In [ ]:
s_test = u"lào quan_trọng hơn việt_nam viettel đang lo cho nước bạn hơn còn trong nước cứ củ_từ chán vãi"
s_test_clean = vi_remove_stop_1char(vi_clean3(vi_strip_text2(s_test)))

print s_test
print "CLEAN: " + '-' * 80
print s_test_clean

In [ ]:
vecs = vectorizer_tfidf.transform([s_test_clean])
vec5 = mbk.transform(vecs)
vec20 = mbk2.transform(vecs)
vec50 = mbk50.transform(vecs)

print vecs[0]
print "Kmean 5: " + '-' * 80
print vec5[0]
print "Kmean 20: " + '-' * 80
print vec20[0]
print "Kmean 50: " + '-' * 80
print vec50[0]

print '-' * 80
print "K5: {}, K20: {}, K50: {}".format(mbk.predict(vecs)[0], mbk2.predict(vecs)[0], mbk50.predict(vecs)[0] )

## KNN distance weight

In [ ]:
from sklearn import neighbors

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=11, weights='distance')

## Clustering metrics

In [ ]:
from sklearn import metrics

In [ ]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels, sample_size=1000))

print()

In [ ]:
 print("Top terms per cluster:")
order_centroids = mbk2.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer_tfidf.get_feature_names()
for i in range(20):
    print("Cluster %d:" % i)
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind])
    print '.' * 80

In [ ]:
mbk2.cluster_centers_.shape

In [ ]:
print vec20[0]

In [ ]:
print np.argsort(vec20[0])
print 'REVERSED: ' + '-' * 80
print np.argsort(vec20[0])[::-1]

## Sanity check

In [ ]:
predictions = []
with codecs.open("/home/laampt/nlp/data/feeds/tok/good_feeds.txt", encoding="utf-8", mode="r") as fin:
    for line in fin:
        s_test_clean = vi_remove_stop_1char(vi_clean3(vi_strip_text2(line)))
        vecs = vectorizer_tfidf.transform([s_test_clean])
        prediction2 = mbk2.predict(vecs)[0]
        prediction50 = mbk50.predict(vecs)[0]
        if prediction: predictions.append(prediction)
        print s_test_clean
        print prediction2, prediction50
        print 

In [ ]:
for ii, label in enumerate(mbk2.labels_):
    print feed_clean[ii], label
    print 

## EDA

In [14]:
%matplotlib inline 

from IPython.display import Image
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
from tsne import tsne # See http://lvdmaaten.github.io/tsne/#implementations

from sklearn import cross_validation
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import metrics
from sklearn import preprocessing
from sklearn import tree

plt.style.use('fivethirtyeight') # Good looking plots
pd.set_option('display.max_columns', None) # Display any number of columns

import seaborn as sns

In [ ]:
feed_df = pd.DataFrame(feed_clean, columns=['Feed'])

In [ ]:
feed_df['C5'] = mbk.labels_
feed_df['C20'] = mbk2.labels_
feed_df['C50'] = mbk50.labels_

In [ ]:
feed_df.head()

In [ ]:
feed_df.hist(bins=20)
plt.show()

In [ ]:
feed_df[feed_df.C20 == 10][feed_df.C50 == 35].head()

In [ ]:
feed_df[feed_df.C20 == 11].head(20)

In [ ]:
print feed_df.ix[13]['Feed']

In [ ]:
feed_df[feed_df.C20 == 16].head(20)

In [ ]:
feed_df[feed_df.C20 == 12][feed_df.C50 == 46].head(10)

## Mega predict

$$ predict(s) = mbk2.predict(s) + mbk50.predict(s) $$

prediction = 1 if:

- c20 == 10 & c50 == 35 
- c20 == 11 & c50 == 31
- c20 == 12 & c50 == 32
- c20 == 16 & c50 == 9 

In [ ]:
def mbk_prediction(feed, vectorizer_tfidf, mbk2, mbk50):
    """
    Input: given raw feed, vectorizer, kmean 20 clusters, kmean 50 clusters
    Output: 1 if feed is user opinion alike else 0
    """
    s_test_clean = vi_remove_stop_1char(vi_clean3(vi_strip_text2(feed)))
    vecs = vectorizer_tfidf.transform([s_test_clean])
    p20 = mbk2.predict(vecs)[0]
    p50 = mbk50.predict(vecs)[0]
    
    if (p20 == 10 and p50 == 35) or (p20 == 11 and p50 == 31) or (p20 == 12 and p50 == 32) or (p20 == 16 and p50 == 9):
        prediction = 1
    else:
        prediction = 0
    return prediction

In [ ]:
feed = u"minh gooner chuyển xài viettel gói_cước đi thấy xài good nhá"
s_test_clean = vi_remove_stop_1char(vi_clean3(vi_strip_text2(feed)))
print s_test_clean
vecs = vectorizer_tfidf.transform([s_test_clean])
print vecs[0]
print mbk2

In [ ]:
with codecs.open("/home/laampt/nlp/data/feeds/tok/good_feeds.txt", encoding="utf-8", mode="r") as fin:
    for line in fin:
        prediction = mega_prediction(line, vectorizer_tfidf, mbk2, mbk50)
        
        print line
        print 'RESULT: [{}]'.format(prediction)
        print '.' * 80

## Extract feed values

In [ ]:
sent_feeds = []

sent_feeds += list(feed_df[feed_df.C20 == 10][feed_df.C50 == 35]['Feed'].values)
sent_feeds += list(feed_df[feed_df.C20 == 11][feed_df.C50 == 31]['Feed'].values)
sent_feeds += list(feed_df[feed_df.C20 == 12][feed_df.C50 == 32]['Feed'].values)
sent_feeds += list(feed_df[feed_df.C20 == 16][feed_df.C50 == 9]['Feed'].values)

print len(sent_feeds)

In [ ]:
print sent_feeds[-1]

In [ ]:
with codecs.open("/home/laampt/nlp/data/feeds/tok/sent_feeds.txt", encoding='utf-8', mode='w') as fout:
    for feed in sent_feeds:
        fout.write(feed + '\n')

In [ ]:
with codecs.open("/home/laampt/nlp/data/feeds/tok/fact_feeds.txt", encoding='utf-8', mode='w') as fout:
    for feed in feed_clean:
        if feed:
            prediction = mega_prediction(feed, vectorizer_tfidf, mbk2, mbk50)
            if prediction == 0:
                fout.write(feed + '\n')

## Supervised flow

In [2]:
pos_docs = []

with codecs.open("/home/laampt/nlp/data/feeds/tok/sent_feeds.txt.nodup", encoding="utf-8", mode="r") as fin:
    pos_docs = fin.readlines()
    
print len(pos_docs)

3616


In [3]:
neg_docs = []

with codecs.open("/home/laampt/nlp/data/feeds/tok/fact_feeds.txt.nodup", encoding="utf-8", mode="r") as fin:
    neg_docs = fin.readlines()
    
print len(neg_docs)

20377


In [4]:
pos_docs_clean_io = []
for i, raw in enumerate(pos_docs):
    if i % 100 == 0: print "[*] Progress ", i
    doc = vi_remove_stop_1char(vi_clean3(vi_strip_text2(raw)))
    pos_docs_clean_io.append(doc)

[*] Progress  0
[*] Progress  100
[*] Progress  200
[*] Progress  300
[*] Progress  400
[*] Progress  500
[*] Progress  600
[*] Progress  700
[*] Progress  800
[*] Progress  900
[*] Progress  1000
[*] Progress  1100
[*] Progress  1200
[*] Progress  1300
[*] Progress  1400
[*] Progress  1500
[*] Progress  1600
[*] Progress  1700
[*] Progress  1800
[*] Progress  1900
[*] Progress  2000
[*] Progress  2100
[*] Progress  2200
[*] Progress  2300
[*] Progress  2400
[*] Progress  2500
[*] Progress  2600
[*] Progress  2700
[*] Progress  2800
[*] Progress  2900
[*] Progress  3000
[*] Progress  3100
[*] Progress  3200
[*] Progress  3300
[*] Progress  3400
[*] Progress  3500
[*] Progress  3600


In [5]:
neg_docs_clean_io = []

for i, raw in enumerate(neg_docs):
    if i % 100 == 0: print "[*] Progress ", i
    doc = vi_remove_stop_1char(vi_clean3(vi_strip_text2(raw)))
    neg_docs_clean_io.append(doc)

[*] Progress  0
[*] Progress  100
[*] Progress  200
[*] Progress  300
[*] Progress  400
[*] Progress  500
[*] Progress  600
[*] Progress  700
[*] Progress  800
[*] Progress  900
[*] Progress  1000
[*] Progress  1100
[*] Progress  1200
[*] Progress  1300
[*] Progress  1400
[*] Progress  1500
[*] Progress  1600
[*] Progress  1700
[*] Progress  1800
[*] Progress  1900
[*] Progress  2000
[*] Progress  2100
[*] Progress  2200
[*] Progress  2300
[*] Progress  2400
[*] Progress  2500
[*] Progress  2600
[*] Progress  2700
[*] Progress  2800
[*] Progress  2900
[*] Progress  3000
[*] Progress  3100
[*] Progress  3200
[*] Progress  3300
[*] Progress  3400
[*] Progress  3500
[*] Progress  3600
[*] Progress  3700
[*] Progress  3800
[*] Progress  3900
[*] Progress  4000
[*] Progress  4100
[*] Progress  4200
[*] Progress  4300
[*] Progress  4400
[*] Progress  4500
[*] Progress  4600
[*] Progress  4700
[*] Progress  4800
[*] Progress  4900
[*] Progress  5000
[*] Progress  5100
[*] Progress  5200
[*] P

In [8]:
import numpy as np
from sklearn.externals import joblib

In [9]:
vectorizer_tfidf = joblib.load("/home/laampt/nlp/feedrank/model/feedrank_24Kdoc_vectorizer_tfidf.pkl")

In [10]:
train_docs  = pos_docs_clean_io + neg_docs_clean_io
train_docs_list = [doc.split() for doc in train_docs]

In [11]:
X_train = np.array(train_docs)
y_train = np.hstack((np.ones(len(pos_docs_clean_io)), np.zeros(len(neg_docs_clean_io))))

In [12]:
train_data_features3 = vectorizer_tfidf.transform(train_docs)
train_data_features3 = train_data_features3.toarray()

## CV10

In [13]:
def stratified_cv3(X, y, clf, shuffle=True, n_folds=10, **kwargs):
    stratified_k_fold = cross_validation.StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
    y_pred = y.copy()
    for ii, jj in stratified_k_fold:
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf.fit(X_train,y_train)
        y_pred[jj] = clf.predict(X_test)
    return clf

In [15]:
lr_tfidf10 = stratified_cv3(train_data_features3, y_train, linear_model.LogisticRegression(C=100))
svc_tfidf10 = stratified_cv3(train_data_features3, y_train, svm.LinearSVC(C=1))

In [16]:
print 'Logistic Regression:\n{}'.format(metrics.classification_report(y_train, lr_tfidf10.predict(train_data_features3)))
print 'Support vector machine(SVM):\n {}'.format(metrics.classification_report(y_train, svc_tfidf10.predict(train_data_features3)))

Logistic Regression:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     20377
        1.0       0.98      0.98      0.98      3616

avg / total       0.99      0.99      0.99     23993

Support vector machine(SVM):
              precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     20377
        1.0       0.97      0.96      0.96      3616

avg / total       0.99      0.99      0.99     23993



In [17]:
lr_weights = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
mega_predictions = []
for w in lr_weights:
    print "LR: {}, SVC: {}".format(w,1-w)
    mega_predictions = []
    for vec in train_data_features3:
        prediction = lr_tfidf10.predict(vec) * w + svc_tfidf10.predict(vec) * (1-w)
        prediction = 1 if (prediction[0] > 0.5) else 0
        mega_predictions.append(prediction)
    print 'Mega Combinator:\n{}'.format(metrics.classification_report(y_train, mega_predictions))
    print('Mega Classifier: {:.2f} AUC'.format(metrics.roc_auc_score(y_train, mega_predictions)))
    print

LR: 0.1, SVC: 0.9
Mega Combinator:
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     20377
        1.0       0.97      0.96      0.96      3616

avg / total       0.99      0.99      0.99     23993

Mega Classifier: 0.98 AUC

LR: 0.2, SVC: 0.8
Mega Combinator:
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     20377
        1.0       0.97      0.96      0.96      3616

avg / total       0.99      0.99      0.99     23993

Mega Classifier: 0.98 AUC

LR: 0.3, SVC: 0.7
Mega Combinator:
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     20377
        1.0       0.97      0.96      0.96      3616

avg / total       0.99      0.99      0.99     23993

Mega Classifier: 0.98 AUC

LR: 0.4, SVC: 0.6
Mega Combinator:
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99     20377
        1.0       0.97     

In [18]:
joblib.dump(lr_tfidf10, "/home/laampt/nlp/feedrank/estimator/39/lr_tfidf_99auc_10cv2.pkl")
joblib.dump(svc_tfidf10, "/home/laampt/nlp/feedrank/estimator/39/svc_tfidf_99auc_10cv2.pkl")

['/home/laampt/nlp/feedrank/estimator/39/svc_tfidf_99auc_10cv2.pkl',
 '/home/laampt/nlp/feedrank/estimator/39/svc_tfidf_99auc_10cv2.pkl_01.npy',
 '/home/laampt/nlp/feedrank/estimator/39/svc_tfidf_99auc_10cv2.pkl_02.npy',
 '/home/laampt/nlp/feedrank/estimator/39/svc_tfidf_99auc_10cv2.pkl_03.npy']

In [ ]:
def mega_prediction2(feed, vectorizer_tfidf, lr_tfidf10, svc_tfidf10):
    """
    Input: given raw feed, vectorizer, lr, svm classifier
    Output: 1 if feed is user opinion alike else 0
    """
    s_test_clean = vi_remove_stop_1char(vi_clean3(vi_strip_text2(feed)))
    vec = vectorizer_tfidf.transform([s_test_clean])[0]
    prediction = lr_tfidf10.predict(vec) * w + svc_tfidf10.predict(vec) * (1-w)
    prediction = 1 if (prediction[0] > 0.5) else 0
    return prediction

## Sanity test

In [ ]:
pos1 = u"vt làm_việc chuyên_nghiệp và nhiệt_tình phết _wwNUMBERww_ mình tới đây lần nào cũng hài_lòng"
neg2 = u"chương_trình khuyến_mãi thẻ_cào nhân dịp quốc_khánh wwDATEww Nạp wwNUMBERww được wwNUMBERww ông chú vt bảo vậy"

pred_mbk_pos1 = mbk_prediction(pos1, vectorizer_tfidf, mbk2, mbk50)
pred_mbk_neg2 = mbk_prediction(neg2, vectorizer_tfidf, mbk2, mbk50)

pred_mega2_pos1 = mega_prediction2(pos1, vectorizer_tfidf, lr_tfidf10, svc_tfidf10)
pred_mega2_neg2 = mega_prediction2(neg2, vectorizer_tfidf, lr_tfidf10, svc_tfidf10)

print "REPORT"
print "-" * 80

print "POS: "
print pos1
print "PREDICT: MBK [{}] MEGA [{}]".format(pred_mbk_pos1, pred_mega2_pos1)
print "." * 80

print "NEG: "
print neg2
print "PREDICT: MBK [{}] MEGA [{}]".format(pred_mbk_neg2, pred_mega2_neg2)
print "." * 80